In [1]:
import set_paths

In [2]:
from make_data import make_cookies_observations
from probability.discrete.discrete import Discrete
from probability.calculations.bayes_rule import MultipleBayesRule

## 1.3 The cookie problem

In [3]:
cookie_data = make_cookies_observations()

In [4]:
cookie_data

,bowl,flavor
0,bowl 1,vanilla
1,bowl 1,vanilla
2,bowl 1,vanilla
3,bowl 1,vanilla
4,bowl 1,vanilla
...,...,...
75,bowl 2,chocolate
76,bowl 2,chocolate
77,bowl 2,chocolate
78,bowl 2,chocolate


In [5]:
cookie_data.value_counts().rename('count').reset_index()

,bowl,flavor,count
0,bowl 1,vanilla,30
1,bowl 2,vanilla,20
2,bowl 2,chocolate,20
3,bowl 1,chocolate,10


In [6]:
cookies = Discrete.from_observations(cookie_data)

In [7]:
cookies.data.reset_index()

,bowl,flavor,"p(bowl, flavor)"
0,bowl 1,chocolate,0.125
1,bowl 1,vanilla,0.375
2,bowl 2,chocolate,0.250
3,bowl 2,vanilla,0.250


### p(vanilla | bowl 1)

In [8]:
vanilla__bowl_1 = cookies.given(bowl='bowl 1').p(flavor='vanilla')

In [9]:
vanilla__bowl_1

0.75

## 1.4 Bayes's theorem

### p(vanilla)

In [10]:
vanilla = cookies.p(flavor='vanilla')

In [11]:
vanilla

0.625

### p(bowl)

In [12]:
bowl = Discrete.from_probs({'bowl 1': 0.5, 'bowl 2': 0.5}, variables=['bowl'])

In [13]:
bowl.data.reset_index()

,bowl,p(bowl)
0,bowl 1,0.5
1,bowl 2,0.5


### p(bowl 1)

In [14]:
bowl_1 = bowl.p(bowl='bowl 1')

In [15]:
bowl_1

0.5

### p(bowl 1 | vanilla)

In [16]:
bowl_1__vanilla = bowl_1 * vanilla__bowl_1 / vanilla

In [17]:
bowl_1__vanilla

0.6

#### check

In [18]:
cookies.given(flavor='vanilla').p(bowl='bowl 1')

0.6

## 1.5 The diachronic interpretation

### p(bowl 2)

In [19]:
bowl_2 = bowl.p(bowl='bowl 2')

In [20]:
bowl_2

0.5

### p(vanilla | bowl 2)

In [21]:
vanilla__bowl_2 = cookies.given(bowl='bowl 2').p(flavor='vanilla')

In [22]:
vanilla__bowl_2

0.5

### p(vanilla)

In [23]:
bowl_1 * vanilla__bowl_1 + bowl_2 * vanilla__bowl_2

0.625

In [24]:
vanilla

0.625

In [25]:
cookies.data.reset_index()

,bowl,flavor,"p(bowl, flavor)"
0,bowl 1,chocolate,0.125
1,bowl 1,vanilla,0.375
2,bowl 2,chocolate,0.250
3,bowl 2,vanilla,0.250


## 1.6 The M & M problem

In [26]:
mix_1994 = Discrete.from_probs({
    'brown': 0.3, 'yellow': 0.2, 'red': 0.2,
    'green': 0.1, 'orange': 0.1, 'tan': 0.1
}, variables='color')

In [27]:
mix_1994.data.reset_index()

,color,p(color)
0,brown,0.3
1,yellow,0.2
2,red,0.2
3,green,0.1
4,orange,0.1
5,tan,0.1


In [28]:
mix_1996 = Discrete.from_probs({
    'blue': 0.24, 'green': 0.2, 'orange': 0.16,
    'yellow': 0.14, 'red': 0.13, 'brown': 0.13
}, variables='color')

In [29]:
mix_1996.data.reset_index()

,color,p(color)
0,blue,0.24
1,green,0.20
2,orange,0.16
3,yellow,0.14
4,red,0.13
5,brown,0.13


In [30]:
bag = Discrete.from_probs({1994: 0.5, 1996: 0.5}, variables='bag')

In [31]:
bag.data.reset_index()

,bag,p(bag)
0,1994,0.5
1,1996,0.5


In [32]:
yellow__1994 = mix_1994.p(color='yellow')
yellow__1996 = mix_1996.p(color='yellow')
green__1994 = mix_1994.p(color='green')
green__1996 = mix_1996.p(color='green')

In [33]:
likelihood_a = yellow__1994 * green__1996
likelihood_b = yellow__1996 * green__1994

In [34]:
prior_likelihood_a = bag.p(bag=1994) * likelihood_a
prior_likelihood_b = bag.p(bag=1996) * likelihood_b

In [35]:
evidence = prior_likelihood_a + prior_likelihood_b

In [36]:
prior_likelihood_a / evidence

0.7407407407407407

In [37]:
prior_likelihood_b / evidence

0.25925925925925924

### using BayesRule class

In [38]:
priors = {'a': 0.5, 'b': 0.5}

In [39]:
likelihoods = {
    'a': yellow__1994 * green__1996,
    'b': yellow__1996 * green__1994
}

In [40]:
bayes_rule = MultipleBayesRule(
    prior=priors, likelihood=likelihoods
)

In [41]:
bayes_rule.posterior()

a    0.740741
b    0.259259
dtype: float64

## 1.7 The Monty Hall problem

In [42]:
priors = {'a': 1 / 3, 'b': 1 / 3, 'c': 1 / 3}

In [43]:
likelihoods = {'a': 1 / 2, 'b': 0, 'c': 1}

In [44]:
MultipleBayesRule(prior=priors, likelihood=likelihoods).posterior()

a    0.333333
b    0.000000
c    0.666667
dtype: float64

### variation

In [45]:
likelihoods = {'a': 1, 'b': 0, 'c': 1}

In [46]:
MultipleBayesRule(prior=priors, likelihood=likelihoods).posterior()

a    0.5
b    0.0
c    0.5
dtype: float64